In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from meteostat import Daily, Stations
from tqdm import tqdm
from math import radians

# Loading weather data #
In this notebook we show how we imported the weather data.

warning - This code in this notebook may take exceptionally long time to run.

important note - we slightly modified this notebook to prevent it from saving the imported data to .pkl. When we used it - we actually saved the fetched data to be later used in the main notebook.

In [3]:
# loading stations data
stations = Stations()
stations = stations.region('US')

stations_df = stations.fetch()

stations_df['latitude_radians'] = stations_df['latitude'].apply(radians)
stations_df['longitude_radians'] = stations_df['longitude'].apply(radians)

stations_df.head()

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,latitude_radians,longitude_radians
id,,,,,,,,,,,,,,,,,
04AEH,Norwich,US,NY,<NA>,KOIC,42.5665,-75.5242,312.0,America/New_York,2022-04-23,2023-02-21,2022-04-23,2022-04-26,NaT,NaT,0.742926,-1.318146
0MV8M,Hill Air Force Base,US,UT,<NA>,KHIF,41.1111,-111.9623,1459.0,America/Denver,2022-04-23,2023-02-21,2022-04-23,2022-04-25,NaT,NaT,0.717524,-1.954111
0NNEW,Effingham County Memorial Airport,US,IL,<NA>,K1H2,39.0706,-88.5333,179.0,America/Chicago,2022-05-06,2023-02-20,NaT,NaT,NaT,NaT,0.681911,-1.545198
0OBKP,Live Oak County Airport,US,TX,<NA>,K8T6,28.3628,-98.1165,39.0,America/Chicago,2022-05-06,2023-02-21,NaT,NaT,NaT,NaT,0.495024,-1.712456
0RJDR,Hotel (Gurley),US,NE,<NA>,K1HW,41.3200,-102.8300,1263.0,America/Denver,2022-05-06,2023-02-21,NaT,NaT,NaT,NaT,0.721170,-1.794722


In [4]:
# filling missing value using neighbours
cols_lst = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt']

In [4]:
# loading daily data for each (year, station) pair.
df_lst = []
stations_lst = list(stations_df.index)
missing_lst = []

for year in range(1992, 2016):
    start = datetime(year, 1, 1)
    end = datetime(year, 12, 31)

    for station_num in tqdm(range(len(stations_lst)), desc=f'Loading year {year}'):
        station_id = stations_lst[station_num]
        cur_data = Daily(station_id, start, end)
        cur_daily_df = cur_data.fetch()
        cur_daily_df['station_id'] = station_id

        if cur_daily_df.size == 0:
            missing_lst.append((station_id, year))
        else:
            cur_daily_df.loc[:, cols_lst] = cur_daily_df.loc[:, cols_lst].interpolate()

        df_lst.append(cur_daily_df)

Loading year 2015: 100%|██████████| 2701/2701 [04:42<00:00,  9.56it/s]


In [5]:
met_data = pd.concat(df_lst)

In [8]:
stations_lst = list(met_data['station_id'].unique())

for index in tqdm(range(len(stations_lst)), desc=f'Loading...'):
    cur_station_id = stations_lst[index]
    met_data.loc[met_data['station_id'] == cur_station_id, :] = met_data.loc[met_data['station_id'] == cur_station_id, :].interpolate()

Loading...: 100%|██████████| 2050/2050 [50:37<00:00,  1.48s/it]


Imputing remaining missing values.

First by aggregating over (station, month), then if not successful - by (station, year).

In [9]:
met_data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,station_id
time,,,,,,,,,,,
1992-01-01,14.0,6.7,20.6,0.0,0.0,NaN,4.7,18.4,1017.60,NaN,69014
1992-01-02,16.9,12.2,25.0,0.0,0.0,NaN,5.4,25.9,1013.20,NaN,69014
1992-01-03,13.7,11.1,15.6,6.9,0.0,NaN,4.3,20.5,1012.55,NaN,69014
1992-01-04,14.8,11.1,19.4,0.0,0.0,NaN,2.2,13.0,1011.90,NaN,69014
1992-01-05,12.6,10.0,15.0,30.7,0.0,NaN,12.2,53.6,1007.70,NaN,69014
...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,NaN,-0.6,6.1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,U9ANI
2015-12-28,NaN,-2.8,3.9,0.0,0.0,NaN,NaN,NaN,NaN,NaN,U9ANI
2015-12-29,NaN,-2.8,6.7,7.4,0.0,NaN,NaN,NaN,NaN,NaN,U9ANI


In [15]:
met_data.reset_index(None, inplace=True)

met_data.head()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,station_id
0,1992-01-01,14.0,6.7,20.6,0.0,0.0,NaN,4.7,18.4,1017.60,NaN,69014
1,1992-01-02,16.9,12.2,25.0,0.0,0.0,NaN,5.4,25.9,1013.20,NaN,69014
2,1992-01-03,13.7,11.1,15.6,6.9,0.0,NaN,4.3,20.5,1012.55,NaN,69014
3,1992-01-04,14.8,11.1,19.4,0.0,0.0,NaN,2.2,13.0,1011.90,NaN,69014
4,1992-01-05,12.6,10.0,15.0,30.7,0.0,NaN,12.2,53.6,1007.70,NaN,69014


In [16]:
met_data['month'] = met_data['time'].apply(lambda x: x.month)
met_data['year'] = met_data['time'].apply(lambda x: x.year)

met_data.head()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,station_id,month,year
0,1992-01-01,14.0,6.7,20.6,0.0,0.0,NaN,4.7,18.4,1017.60,NaN,69014,1,1992
1,1992-01-02,16.9,12.2,25.0,0.0,0.0,NaN,5.4,25.9,1013.20,NaN,69014,1,1992
2,1992-01-03,13.7,11.1,15.6,6.9,0.0,NaN,4.3,20.5,1012.55,NaN,69014,1,1992
3,1992-01-04,14.8,11.1,19.4,0.0,0.0,NaN,2.2,13.0,1011.90,NaN,69014,1,1992
4,1992-01-05,12.6,10.0,15.0,30.7,0.0,NaN,12.2,53.6,1007.70,NaN,69014,1,1992


In [17]:
agg_df = met_data.groupby(['station_id', 'month']).mean()

agg_df

tavg       tmin       tmax      prcp      snow  \
station_id month                                                        
69014      1      13.998848   9.541935  19.196774  4.340092  0.000000   
           2      14.251852   9.829798  19.334848  5.184343  0.000000   
           3      15.311674  10.752074  20.851613  3.005991  0.000000   
           4      16.858095  11.695714  22.914762  0.435238  0.000000   
           5      18.140092  14.094009  23.281106  0.298157  0.000000   
...                     ...        ...        ...       ...       ...   
U9ANI      8            NaN  15.763668  30.289594  0.504762  0.000000   
           9            NaN  12.920751  27.821825  0.352057  0.000000   
           10           NaN   8.534192  21.767140  1.518117  0.000000   
           11           NaN   5.236287  16.012500  2.172295  0.046642   
           12           NaN   2.254766  11.559712  3.357464  6.771583   

                        wdir      wspd       wpgt         pres  tsun  \
station_id month                                                       
69014      1      101.665140  5.963134  22.643318  1016.910876   NaN   
           2      128.822172  6.721212  26.274242  1016.737222   NaN   
           3      148.345229  6.158525  23.903226  1015.923018   NaN   
           4      162.970436  6.175238  24.290952  1014.395952   NaN   
           5      165.337185  5.993548  23.080645  1013.609793   NaN   
...                      ...       ...        ...          ...   ...   
U9ANI      8             NaN       NaN        NaN          NaN   NaN   
           9             NaN       NaN        NaN          NaN   NaN   
           10            NaN       NaN        NaN          NaN   NaN   
           11            NaN       NaN        NaN          NaN   NaN   
           12            NaN       NaN        NaN          NaN   NaN   

                         year  
station_id month               
69014      1      1995.000000  
           2      1994.989899  
           3      1995.000000  
           4      1995.000000  
           5      1995.000000  
...                       ...  
U9ANI      8      2004.523810  
           9      2004.622540  
           10     2004.550622  
           11     2004.927239  
           12     2004.973022  

[24304 rows x 11 columns]

In [20]:
for col_name in cols_lst:
    sub_df = met_data.loc[met_data[col_name].isna(), ['station_id', 'month', col_name]]

    if len(sub_df) == 0:
        continue

    sub_df[col_name] = list(zip(sub_df.station_id, sub_df.month))
    sub_df[col_name] = sub_df[col_name].apply(lambda x: agg_df.loc[x, col_name])

    met_data.loc[sub_df.index, col_name] = sub_df.loc[:, col_name]

In [26]:
agg_df = met_data.groupby(['station_id', 'year']).mean()

agg_df

tavg       tmin       tmax      prcp       snow  \
station_id year                                                         
69014      1992  18.967486  14.245902  24.869945  1.616120   0.000000   
           1993  18.302466  13.644110  24.106027  1.750411   0.000000   
           1994  17.806986  13.131507  23.860822  0.807945   0.000000   
           1995  17.065479  12.495890  22.837260  1.829315   0.000000   
           1996  17.526230  12.996721  23.097541  1.118306   0.000000   
...                    ...        ...        ...       ...        ...   
U9ANI      2011        NaN   8.489315  18.913699  1.593699  13.150685   
           2012        NaN   9.605738  20.486339  1.253552   8.658470   
           2013        NaN   9.247945  19.858082  1.118630   1.252055   
           2014        NaN  10.507397  21.314521  1.167671  10.038356   
           2015        NaN   9.765479  20.287671  1.666301   1.112329   

                       wdir      wspd       wpgt         pres  tsun     month  
station_id year                                                                
69014      1992  114.937497  6.423224  27.762295  1013.633743   NaN  6.513661  
           1993  168.647945  6.037534  24.495890  1013.822466   NaN  6.526027  
           1994  154.771233  5.286301  19.940548  1015.368904   NaN  6.526027  
           1995  200.157534  4.784384  19.142740  1014.736849   NaN  6.526027  
           1996  186.900273  5.054645  16.881148  1015.125137   NaN  6.513661  
...                     ...       ...        ...          ...   ...       ...  
U9ANI      2011         NaN       NaN        NaN          NaN   NaN  6.526027  
           2012         NaN       NaN        NaN          NaN   NaN  6.513661  
           2013         NaN       NaN        NaN          NaN   NaN  6.526027  
           2014         NaN       NaN        NaN          NaN   NaN  6.526027  
           2015         NaN       NaN        NaN          NaN   NaN  6.526027  

[25993 rows x 11 columns]

In [27]:
for col_name in cols_lst:
    sub_df = met_data.loc[met_data[col_name].isna(), ['station_id', 'year', col_name]]

    if len(sub_df) == 0:
        continue

    sub_df[col_name] = list(zip(sub_df.station_id, sub_df.year))
    sub_df[col_name] = sub_df[col_name].apply(lambda x: agg_df.loc[x, col_name])

    met_data.loc[sub_df.index, col_name] = sub_df.loc[:, col_name]

    print(f"imputed column {col_name}")

imputed column tavg
imputed column tmin
imputed column tmax
imputed column prcp
imputed column snow
imputed column wdir
imputed column wspd
imputed column wpgt


In [30]:
new_met_data = met_data.dropna(subset=cols_lst)

new_met_data

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,station_id,month,year
0,1992-01-01,14.0,6.7,20.6,0.0,0.0,101.665140,4.7,18.4,1017.600000,NaN,69014,1,1992
1,1992-01-02,16.9,12.2,25.0,0.0,0.0,101.665140,5.4,25.9,1013.200000,NaN,69014,1,1992
2,1992-01-03,13.7,11.1,15.6,6.9,0.0,101.665140,4.3,20.5,1012.550000,NaN,69014,1,1992
3,1992-01-04,14.8,11.1,19.4,0.0,0.0,101.665140,2.2,13.0,1011.900000,NaN,69014,1,1992
4,1992-01-05,12.6,10.0,15.0,30.7,0.0,101.665140,12.2,53.6,1007.700000,NaN,69014,1,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7687432,2015-12-27,21.3,17.8,26.7,22.4,0.0,195.857143,8.6,25.9,1014.263636,429.0,91285,12,2015
7687433,2015-12-28,22.3,20.0,28.9,20.8,0.0,194.000000,9.4,25.9,1013.800000,429.0,91285,12,2015
7687434,2015-12-29,22.4,17.8,28.3,0.0,0.0,194.000000,6.5,25.9,1012.000000,429.0,91285,12,2015
7687435,2015-12-30,22.6,18.9,27.8,0.0,0.0,194.000000,13.7,25.9,1014.800000,429.0,91285,12,2015


In [35]:
new_stations_df = stations_df.loc[new_met_data['station_id'].unique(), :]

new_stations_df

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,latitude_radians,longitude_radians
id,,,,,,,,,,,,,,,,,
69014,Irvine,US,CA,69014,KNZJ,33.6667,-117.7167,119.0,America/Los_Angeles,1989-01-01,1999-06-06,1945-03-01,1998-12-10,1945-01-01,1998-01-01,0.587595,-2.054544
69015,Twentynine Palms / Sunfair Heights,US,CA,69015,KNXP,34.2962,-116.1622,625.0,America/Los_Angeles,1990-01-02,2023-02-21,1945-05-01,2022-04-24,1945-01-01,2021-01-01,0.598582,-2.027413
69023,Whidbey Island / Ault Field,US,WA,69023,KNUW,48.3518,-122.6560,14.0,America/Los_Angeles,2005-01-01,2023-02-20,1945-04-01,2022-04-24,1945-01-01,2022-01-01,0.843898,-2.140751
70026,Post Rogers Memorial Airport,US,AK,70026,PABR,71.2833,-156.7667,13.0,America/Juneau,1945-01-01,2023-02-21,1901-09-01,2023-02-16,1901-01-01,2022-01-01,1.244128,-2.736095
70133,Wien Memorial Airport,US,AK,70133,PAOT,66.8833,-162.6000,3.0,America/Nome,1973-01-01,2023-02-21,1897-09-12,2023-02-16,1897-01-01,2022-01-01,1.167334,-2.837905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91176,Mcbh Kaneohe Bay Hawaii,US,HI,91176,PHNG,21.4500,-157.7500,7.0,Pacific/Honolulu,1973-01-01,2023-02-21,1942-01-01,2023-02-12,1942-01-01,2022-01-01,0.374373,-2.753257
91182,Honolulu International Airport,US,HI,91182,PHNL,21.3333,-157.9500,4.0,Pacific/Honolulu,1939-06-01,2023-02-21,1939-06-01,2023-02-14,1939-01-01,2022-01-01,0.372336,-2.756748
91190,Kahului Airport,US,HI,91190,PHOG,20.9000,-156.4333,16.0,Pacific/Honolulu,1973-01-01,2023-02-21,1905-01-01,2023-02-12,1905-01-01,2022-01-01,0.364774,-2.730276
